In [2]:
import pandas as pd
import os
from matplotlib import pyplot as plt
import datetime

%matplotlib notebook

from IPython.display import display, HTML


one_drive_folder = "../Incident Prediction"


pd_data_input = "/pagerduty_Incident_data/incidents/"
pd_data_output ="/pagerduty_Incident_data/generated/"

input_directory = one_drive_folder + pd_data_input
output_directory = one_drive_folder + pd_data_output

incidents = pd.DataFrame(columns=['id', 'incident_number', 'description', 'service_id', 'service_name',
                           'escalation_policy_id','escalation_policy_name','created_on', 'resolved_on'
                           'seconds_to_first_ack','seconds_to_resolve', 'auto_resolved', 
                           'escalation_count', 'auto_escalation_count', 'acknowledge_count',
                           'assignment_count', 'acknowledged_by_user_ids','acknowledged_by_user_names', 
                           'assigned_to_user_ids','resolved_by_user_id','urgency'])

#pre-processing of Pagerduty incides data starts here
#loading the Pagerduty incidents data
for root,dirs,files in os.walk(input_directory):
    for file in files:
        if file.find("2019") >= 0:
            temp = pd.read_csv(input_directory + file)
            incidents = incidents.append(temp)
incidents.to_csv(output_directory + "all_2019.csv")


/Users/a071887/pdip/venv/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [3]:
# comnine PD data
all = pd.read_csv(output_directory + "all_2019.csv")
compact = all[['incident_number','created_on','resolved_on','service_name','description','seconds_to_resolve','urgency']].copy()
compact['created_date'] = [datetime.datetime.date(pd.to_datetime(d,utc=True)) for d in compact['created_on']]
compact['created_time'] = [datetime.datetime.time(pd.to_datetime(d,utc=True)) for d in compact['created_on']]
compact['created_datetime'] = [pd.to_datetime(d,utc=True) for d in compact['created_on']]
compact_datetime = compact


In [4]:
#cleanup PD data
temp1 = compact_datetime[~compact_datetime.service_name.str.contains("BCOM")]
temp2 = temp1[~temp1.service_name.str.contains("STARS")]
temp3 = temp2[~temp2.service_name.str.contains("DaaS")]
temp4 = temp3[~temp3.service_name.str.contains("EDPOC")]
temp5 = temp4[~temp4.service_name.str.contains("TEST")]
temp6 = temp5[~temp5.service_name.str.contains("perf tasks")]
temp7 = temp6[~temp6.service_name.str.contains("escalation")]
temp8 = temp7[~temp7.service_name.str.contains("prometheus")]
temp9 = temp8[~temp8.service_name.str.contains("stella")]
temp10 = temp9[~temp9.service_name.str.contains("Stackdriver")]
temp11 = temp10[~temp10.service_name.str.contains("Test")]
temp12 = temp11[~temp11.service_name.str.contains("EMLP")]
temp13 = temp12[~temp12.service_name.str.contains("testing")]
temp14 = temp13[~temp13.service_name.str.contains("Demo")]
temp15 = temp14[~temp14.service_name.str.contains("Default")]
temp16 = temp15[~temp15.service_name.str.contains("Escalation")]
temp17 = temp16[~temp16.service_name.str.contains("Escalations")]
filtered_incidents = temp17[~temp17.service_name.str.contains("Perf")]
filtered_pd_incidents = filtered_incidents
servicenames = filtered_pd_incidents.service_name.unique()
filtered_incidents.to_csv(output_directory + "all_2019_compact.csv")

In [6]:
#Loading JIRA tickets data
jira_data_input = "/JIRA/"
input_directory = one_drive_folder + jira_data_input 
all_jira = pd.read_csv(input_directory + "jira_filtered.csv")
all_jira.set_index('Created')

#pre-process JIRA data
sev_compact = all_jira[['Created','Summary','Description','Custom field (Brand)','Custom field (Channel)','Custom field (Component)', 'Issue key']].copy()
sev_compact['jira_ticket'] = sev_compact['Issue key'].astype(str)
del sev_compact['Issue key']
sev_compact['created_date'] = [datetime.datetime.date(pd.to_datetime(d,utc=True)) for d in sev_compact['Created']]
sev_compact['created_time'] = [datetime.datetime.time(pd.to_datetime(d,utc=True)) for d in sev_compact['Created']]
sev_compact['created_datetime'] = [pd.to_datetime(d,utc=True) for d in sev_compact['Created']]
sev_compact['service_name'] = ['jira_sev1_issue' for d in sev_compact['Created']]
sev_compact['description'] = [sev_compact['Description'] for d in sev_compact['Description']]
sev_compact.set_index('created_date')
sev_compact_datetime = sev_compact
tempj1 = sev_compact_datetime[~sev_compact_datetime['Custom field (Component)'].str.contains("MSP-Order")]
order_to_batch_jira = tempj1[~tempj1['Summary'].str.contains("Orders To Batch")]
order_to_batch_jira_sab = order_to_batch_jira
jira_otb = order_to_batch_jira
del jira_otb['description']
del jira_otb['Description']
del sev_compact_datetime['description']
del sev_compact_datetime['Description']
sev_compact_datetime.to_csv(output_directory + "jira_otb.csv")


In [7]:
import pandas as pd
all_compact = pd.read_csv(output_directory + "all_2019_compact.csv")
jira_otb = pd.read_csv(output_directory + "jira_otb.csv")

def prepare_jira_vs_incidents(jira_issues, pd_incidents, relative_time_window):
    import pandasql as ps
    import numpy as np

    #jira_issues['created_datetime_plus'] = jira_issues.created_datetime + pd.DateOffset(hours=relative_time_window)
    jira_issues['created_datetime_minus'] = jira_issues.created_datetime - pd.DateOffset(hours=relative_time_window)
    
    sqlcode = '''
    select  pd_incidents.incident_number pd_incident_number,
            pd_incidents.service_name pd_service_name, 
            pd_incidents.description pd_description,
            pd_incidents.created_datetime pd_created_datetime,
            jira_issues.created_datetime jira_created_datetime,
            jira_issues.service_name jira_service_name,
            jira_issues.jira_ticket jira_ticket
    from jira_issues
    left outer join pd_incidents on pd_incidents.created_datetime >= jira_issues.created_datetime_minus 
    and pd_incidents.created_datetime <= jira_issues.created_datetime
    '''
    combined = ps.sqldf(sqlcode,locals())
    combined = combined.mask(combined.eq('None')).dropna()
    combined['pd_created_datetime'] = [pd.to_datetime(d,utc=True) for d in combined['pd_created_datetime']]
    combined['jira_created_datetime'] = [pd.to_datetime(d,utc=True) for d in combined['jira_created_datetime']]
    combined['issue_happened'] = '1'
    return combined
    
# call the function by passing pre-processed dataframes
combined = prepare_jira_vs_incidents(sev_compact_datetime,filtered_incidents, 5)
#combined

/Users/a071887/pdip/venv/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [8]:
import pandasql as ps
import numpy as np
combined_1 = combined.set_index('pd_incident_number')
incidents_1 = filtered_incidents.set_index('incident_number')

sqlcode = '''
    select  incidents_1.incident_number incident_number,
            incidents_1.description description,
            incidents_1.created_datetime created_datetime,
            combined_1.issue_happened issue_happened,
            combined_1.jira_created_datetime jira_created_datetime,
            combined_1.jira_ticket jira_ticket
    from incidents_1
    left outer join combined_1 on incidents_1.incident_number = combined_1.pd_incident_number 
    '''
labeled_incidents = ps.sqldf(sqlcode,locals())
labeled_incidents = labeled_incidents.fillna(0).sort_values(by=['created_datetime'])
labeled_incidents.to_csv(output_directory + "labelled_incidents.csv")
#labeled_incidents.groupby(['jira_ticket']).count()['description']
train_set = labeled_incidents.head(80000)
test_temp = labeled_incidents.tail(82539)
dev_set = test_temp[test_temp.jira_ticket == 'DS-887937']
test_set = test_temp[test_temp.jira_ticket != 'DS-887937']
train_set.to_csv(output_directory + "train_set.csv")
test_set.to_csv(output_directory + "test_set.csv")
dev_set.to_csv(output_directory + "dev_set.csv")
#train_set
#test_set
dev_set

,incident_number,description,created_datetime,issue_happened,jira_created_datetime,jira_ticket
158199,1649546,Splunk Alert: SEV3 :MCOM:DAL:SNB:mcom_homepage...,2019-11-27 05:15:22.000000,1,2019-11-27 10:15:00.000000,DS-887937
158200,1649547,[FIRING:11] pubsub_topicUnackedMessagesTooHigh...,2019-11-27 05:16:33.000000,1,2019-11-27 10:15:00.000000,DS-887937
158201,1649551,WARNING: 'Cpu-Usage' on 'ma412mlvfcc102',2019-11-27 05:18:10.000000,1,2019-11-27 10:15:00.000000,DS-887937
158202,1649559,Splunk Alert: SEV2:All:WDC:HUB:All:Origin_4XX ...,2019-11-27 05:29:42.000000,1,2019-11-27 10:15:00.000000,DS-887937
158203,1649562,[FIRING:12] pubsub_topicUnackedMessagesTooHigh...,2019-11-27 05:32:34.000000,1,2019-11-27 10:15:00.000000,DS-887937
...,...,...,...,...,...,...
158261,1649703,Re: Splunk Alert: SEV2:MCOM:DAL:ALL:DISCOVERY:...,2019-11-27 09:49:10.000000,1,2019-11-27 10:15:00.000000,DS-887937
158262,1649705,Splunk Alert: SEV1:PLATFORM:DAL:ALL:ALL:SOCKET...,2019-11-27 09:59:25.000000,1,2019-11-27 10:15:00.000000,DS-887937
158263,1649706,Splunk Alert: SEV2:All:WDC:HUB:All:Origin_4XX ...,2019-11-27 09:59:41.000000,1,2019-11-27 10:15:00.000000,DS-887937
158264,1649710,[FIRING:13] pubsub_topicUnackedMessagesTooHigh...,2019-11-27 10:01:33.000000,1,2019-11-27 10:15:00.000000,DS-887937


In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

In [10]:
#train_set = pd.read_csv(output_directory + "train_set.csv")
#test_set = pd.read_csv(output_directory + "test_set.csv")
train_X = train_set['description'].values
train_y = train_set['issue_happened'].values
test_X = test_set['description'].values
test_y = test_set['issue_happened'].values
dev_X = dev_set['description'].values
dev_y = dev_set['issue_happened'].values

In [11]:
vocab_size = 10000
train_encoded_docs = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in train_X]
test_encoded_docs = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in test_X]
dev_encoded_docs = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in dev_X]
train_labels = np.asarray(train_y, dtype=np.uint8)
test_labels = np.asarray(test_y, dtype=np.uint8)
dev_labels = np.asarray(dev_y, dtype=np.uint8)
#train_labels
#test_labels
#train_encoded_docs
#test_encoded_docs
#print(encoded_docs)

In [18]:
# pad documents to a max length of 4 words
max_length = 64
train_padded_docs = tf.keras.preprocessing.sequence.pad_sequences(train_encoded_docs, maxlen=max_length, padding='post')
test_padded_docs = tf.keras.preprocessing.sequence.pad_sequences(test_encoded_docs, maxlen=max_length, padding='post')
dev_padded_docs = tf.keras.preprocessing.sequence.pad_sequences(dev_encoded_docs, maxlen=max_length, padding='post')
#train_padded_docs
#test_padded_docs

In [19]:
sequence_size = 64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, sequence_size, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(sequence_size)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(sequence_size, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [15]:
history = model.fit(train_padded_docs,train_labels, epochs=10)

Train on 80000 samples
Epoch 1/10
80000/80000 [==============================] - 78s 980us/sample - loss: 0.1161 - accuracy: 0.9785
Epoch 2/10
80000/80000 [==============================] - 74s 928us/sample - loss: 0.0928 - accuracy: 0.9789
Epoch 3/10
80000/80000 [==============================] - 74s 930us/sample - loss: 0.0897 - accuracy: 0.9796
Epoch 4/10
80000/80000 [==============================] - 74s 927us/sample - loss: 0.0872 - accuracy: 0.9801
Epoch 5/10
80000/80000 [==============================] - 74s 925us/sample - loss: 0.0852 - accuracy: 0.9803
Epoch 6/10
80000/80000 [==============================] - 74s 925us/sample - loss: 0.0837 - accuracy: 0.9807
Epoch 7/10
80000/80000 [==============================] - 74s 925us/sample - loss: 0.0827 - accuracy: 0.9814
Epoch 8/10
80000/80000 [==============================] - 74s 924us/sample - loss: 0.0820 - accuracy: 0.9814
Epoch 9/10
80000/80000 [==============================] - 74s 923us/sample - loss: 0.0817 - accuracy: 0.9

In [16]:
test_loss, test_acc = model.evaluate(test_padded_docs,test_labels)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

82472/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test Loss: 0.19457168566086175
Test Accuracy: 0.9575734734535217


In [17]:
predictions = model.predict(dev_padded_docs)
print (predictions)

[[0.01107493]
 [0.04007804]
 [0.13256824]
 [0.00905001]
 [0.02265912]
 [0.00905001]
 [0.01255497]
 [0.00905001]
 [0.01061037]
 [0.00905001]
 [0.01123083]
 [0.00687873]
 [0.06638852]
 [0.00905001]
 [0.01263615]
 [0.00362611]
 [0.00017157]
 [0.00017157]
 [0.00440484]
 [0.0075615 ]
 [0.01680896]
 [0.02265912]
 [0.00269663]
 [0.01507223]
 [0.00031459]
 [0.00032547]
 [0.01560301]
 [0.01043946]
 [0.00176579]
 [0.01140815]
 [0.0127278 ]
 [0.02473396]
 [0.01188776]
 [0.04007804]
 [0.00905001]
 [0.01509658]
 [0.00905001]
 [0.01560301]
 [0.01188776]
 [0.01304778]
 [0.00908259]
 [0.00905001]
 [0.01680896]
 [0.0127278 ]
 [0.00051504]
 [0.02265912]
 [0.00905001]
 [0.01680896]
 [0.00984249]
 [0.13256824]
 [0.01680896]
 [0.00616854]
 [0.13256824]
 [0.01780427]
 [0.00905001]
 [0.02196935]
 [0.00033581]
 [0.00192091]
 [0.01509658]
 [0.01188776]
 [0.00085324]
 [0.01175296]
 [0.00105342]
 [0.01304778]
 [0.00905005]
 [0.01507222]
 [0.00905005]]
